In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    FFF_Train_Decile5 = pd.read_pickle("FFF_Train_Decile5.pkl")
    FFF_Train_Targets_1D = pd.read_pickle("FFF_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("FFF_Test_Decile5.pkl")
    y_test = pd.read_pickle("FFF_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(FFF_Train_Decile5, FFF_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150, 200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM FFF Decile 5')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3055/3055 [==============================] - 4s 1ms/step - loss: 0.4001 - acc: 0.8331 - val_loss: 0.3693 - val_acc: 0.8560
Epoch 39/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3997 - acc: 0.8347 - val_loss: 0.3735 - val_acc: 0.8560
Epoch 40/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3958 - acc: 0.8360 - val_loss: 0.3658 - val_acc: 0.8586
Epoch 41/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3964 - acc: 0.8334 - val_loss: 0.3685 - val_acc: 0.8482
Epoch 42/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3998 - acc: 0.8354 - val_loss: 0.3701 - val_acc: 0.8560
Epoch 43/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3954 - acc: 0.8347 - val_loss: 0.3715 - val_acc: 0.8521
Epoch 44/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3941 - acc: 0.8354 - val_loss: 0.3654 - val_acc: 0.8547
Epoch 45/100
3055/3055 [==============================] - 4s 

Epoch 98/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3894 - acc: 0.8347 - val_loss: 0.3611 - val_acc: 0.8599
Epoch 99/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3892 - acc: 0.8393 - val_loss: 0.3614 - val_acc: 0.8599
Epoch 100/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3932 - acc: 0.8337 - val_loss: 0.3620 - val_acc: 0.8586
Test accuracy: 0.8586387431434311
Train on 3055 samples, validate on 764 samples
Epoch 1/150
3055/3055 [==============================] - 5s 2ms/step - loss: 0.6066 - acc: 0.6462 - val_loss: 0.4898 - val_acc: 0.7264
Epoch 2/150
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4914 - acc: 0.7810 - val_loss: 0.4471 - val_acc: 0.8325
Epoch 3/150
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4602 - acc: 0.8187 - val_loss: 0.4216 - val_acc: 0.8272
Epoch 4/150
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4307 - acc: 0.8288 - val_l

3055/3055 [==============================] - 4s 1ms/step - loss: 0.3638 - acc: 0.8422 - val_loss: 0.3387 - val_acc: 0.8429
Epoch 57/150
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3622 - acc: 0.8403 - val_loss: 0.3387 - val_acc: 0.8429
Epoch 58/150
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3630 - acc: 0.8416 - val_loss: 0.3386 - val_acc: 0.8429
Epoch 59/150
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3592 - acc: 0.8422 - val_loss: 0.3384 - val_acc: 0.8429
Epoch 60/150
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3708 - acc: 0.8409 - val_loss: 0.3383 - val_acc: 0.8429
Epoch 61/150
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3604 - acc: 0.8442 - val_loss: 0.3384 - val_acc: 0.8429
Epoch 62/150
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3627 - acc: 0.8439 - val_loss: 0.3382 - val_acc: 0.8429
Epoch 63/150
3055/3055 [==============================] - 4s 

3055/3055 [==============================] - 4s 1ms/step - loss: 0.3636 - acc: 0.8429 - val_loss: 0.3377 - val_acc: 0.8429
Epoch 117/150
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3579 - acc: 0.8448 - val_loss: 0.3376 - val_acc: 0.8429
Epoch 118/150
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3609 - acc: 0.8439 - val_loss: 0.3375 - val_acc: 0.8429
Epoch 119/150
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3645 - acc: 0.8442 - val_loss: 0.3375 - val_acc: 0.8429
Epoch 120/150
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3610 - acc: 0.8426 - val_loss: 0.3375 - val_acc: 0.8429
Epoch 121/150
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3674 - acc: 0.8442 - val_loss: 0.3374 - val_acc: 0.8416
Epoch 122/150
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3603 - acc: 0.8462 - val_loss: 0.3375 - val_acc: 0.8429
Epoch 123/150
3055/3055 [==============================

...

Epoch 66/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4053 - acc: 0.8383 - val_loss: 0.3753 - val_acc: 0.8429
Epoch 67/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4012 - acc: 0.8376 - val_loss: 0.3743 - val_acc: 0.8429
Epoch 68/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4018 - acc: 0.8367 - val_loss: 0.3741 - val_acc: 0.8429
Epoch 69/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4021 - acc: 0.8386 - val_loss: 0.3737 - val_acc: 0.8416
Epoch 70/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3997 - acc: 0.8340 - val_loss: 0.3738 - val_acc: 0.8429
Epoch 71/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4058 - acc: 0.8347 - val_loss: 0.3742 - val_acc: 0.8429
Epoch 72/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4031 - acc: 0.8340 - val_loss: 0.3747 - val_acc: 0.8429
Epoch 73/100
3055/3055 [========================

Epoch 26/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3604 - acc: 0.8409 - val_loss: 0.3623 - val_acc: 0.8482
Epoch 27/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3664 - acc: 0.8455 - val_loss: 0.3357 - val_acc: 0.8560
Epoch 28/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3578 - acc: 0.8498 - val_loss: 0.3331 - val_acc: 0.8599
Epoch 29/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3653 - acc: 0.8439 - val_loss: 0.3384 - val_acc: 0.8534
Epoch 30/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3600 - acc: 0.8426 - val_loss: 0.3361 - val_acc: 0.8599
Epoch 31/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3624 - acc: 0.8462 - val_loss: 0.3317 - val_acc: 0.8547
Epoch 32/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3657 - acc: 0.8465 - val_loss: 0.3373 - val_acc: 0.8560
Epoch 33/200
3055/3055 [========================

3055/3055 [==============================] - 4s 1ms/step - loss: 0.3484 - acc: 0.8498 - val_loss: 0.3267 - val_acc: 0.8547
Epoch 85/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3466 - acc: 0.8501 - val_loss: 0.3267 - val_acc: 0.8560
Epoch 86/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3435 - acc: 0.8514 - val_loss: 0.3266 - val_acc: 0.8560
Epoch 87/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3459 - acc: 0.8520 - val_loss: 0.3266 - val_acc: 0.8560
Epoch 88/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3498 - acc: 0.8511 - val_loss: 0.3267 - val_acc: 0.8560
Epoch 89/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3518 - acc: 0.8527 - val_loss: 0.3269 - val_acc: 0.8560
Epoch 90/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3444 - acc: 0.8570 - val_loss: 0.3268 - val_acc: 0.8560
Epoch 91/200
3055/3055 [==============================] - 4s 

3055/3055 [==============================] - 4s 1ms/step - loss: 0.3458 - acc: 0.8530 - val_loss: 0.3265 - val_acc: 0.8573
Epoch 145/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3489 - acc: 0.8537 - val_loss: 0.3267 - val_acc: 0.8560
Epoch 146/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3485 - acc: 0.8556 - val_loss: 0.3267 - val_acc: 0.8573
Epoch 147/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3461 - acc: 0.8511 - val_loss: 0.3266 - val_acc: 0.8573
Epoch 148/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3470 - acc: 0.8540 - val_loss: 0.3266 - val_acc: 0.8560
Epoch 149/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3494 - acc: 0.8540 - val_loss: 0.3266 - val_acc: 0.8573
Epoch 150/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3505 - acc: 0.8517 - val_loss: 0.3266 - val_acc: 0.8573
Epoch 151/200
3055/3055 [==============================

3055/3055 [==============================] - 4s 1ms/step - loss: 0.4993 - acc: 0.7421 - val_loss: 0.4535 - val_acc: 0.8154
Epoch 4/50
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4736 - acc: 0.8056 - val_loss: 0.4295 - val_acc: 0.8377
Epoch 5/50
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4570 - acc: 0.8105 - val_loss: 0.4092 - val_acc: 0.8298
Epoch 6/50
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4352 - acc: 0.8193 - val_loss: 0.3906 - val_acc: 0.8351
Epoch 7/50
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4153 - acc: 0.8272 - val_loss: 0.3806 - val_acc: 0.8521
Epoch 8/50
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4131 - acc: 0.8255 - val_loss: 0.3892 - val_acc: 0.8416
Epoch 9/50
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4038 - acc: 0.8357 - val_loss: 0.3715 - val_acc: 0.8325
Epoch 10/50
3055/3055 [==============================] - 4s 1ms/step - lo

In [3]:
0.93484

0.93484

In [ ]:
#0.865